# Code for Lecture 3

In [1]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import xlrd

### Simple LP with Gurobi+Python -  First Example 

In [2]:
try:

    # Create a new model
    m = Model("simplelp")

    # Create variables
    x1 = m.addVar(ub = 10, name="x1")
    x2 = m.addVar(name="x2")
    x3 = m.addVar(name="x3")

    # Set objective
    m.setObjective(x1 + 2 * x2 + 5 * x3, GRB.MAXIMIZE)

    # Add constraint: 
    m.addConstr(-x1 + x2 + 3*x3 <= -5, "c0")

    # Add constraint: 
    m.addConstr(x1 + 3*x2 - 7*x3 >= 10, "c1")

    m.optimize()
    
    # print optimal solutions
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    # print optimal value
    print('Obj: %g' % m.objVal)
    
    # print dual values to all constraints
    print(m.getAttr("Pi", m.getConstrs()))

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')


Academic license - for non-commercial use only - expires 2021-03-20
Using license file /Users/bchaitha/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x2dda9a56
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [5e+00, 1e+01]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.000000000e+01
x1 10
x2 5
x3 0
Obj: 20
[2.0, 0.0]


### Alternative way

In [6]:
# Create a new model
m = Model("simplelp")

# Create variables
ubd = [GRB.INFINITY, GRB.INFINITY]
x = m.addVars(2, ub = ubd, name="x")
    
# Set objective
c = [500, 450]
    
m.setObjective(  sum(x[i] * c[i] for i in range(2)) , GRB.MAXIMIZE)

# Add constraints: 
b = [60, 15000, 8]
A = np.array([[6, 5], [1000, 2000], [1,0]])
    
m.addConstrs( quicksum(A[i,j] * x[j] for j in range(2)) <= b[i] for i in range(3))
        
m.optimize()
    
# print optimal solutions
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

# print optimal value
print('Obj: %g' % m.objVal)

# print dual values to all constraints
print(m.getAttr("Pi", m.getConstrs()))


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0x098a2d60
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+04]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4000000e+03   5.687060e+02   0.000000e+00      0s
       2    5.1428571e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  5.142857143e+03
x[0] 6.42857
x[1] 4.28571
Obj: 5142.86
[78.57142857142857, 0.02857142857142858, 0.0]


###  Manufacturing Problem

In [ ]:
# Preparing parameters
c = [5 ,6 ,8, 1, 3]

# available units of material i
b = [20,40,50]


# units of material i product j needs in order to be produced
a = np.array([[6, 7, 9, 8, 5], 
              [7, 4, 1, 9, 6], 
              [8, 3, 5, 7, 2]])
# get size of the problem
n, m = np.shape(a)

# Preparing an optimization model
mm = Model('manufacturing')

# Declaring variables
var = mm.addVars(m)

# Setting the objective
mm.setObjective(sum(var[i]*c[i] for i in range(m)), GRB.MAXIMIZE)

# Adding constraints
mm.addConstrs(quicksum(a[i,j]* var[j] for j in range(m)) <= b[i] for i in range(n))

# Solving the optimization problem
mm.optimize()

# Printing the optimal solutions obtained
p = mm.getAttr('x', var)

# Get objective value for future usage
print("\nOptimal value: \t%g" % mm.objVal)

# alternative
for i, v in var.items(): 
    print("\nproduce product %g amount: %g" % (i, v.getAttr("x")))


In [ ]:
# Manufacturing Problem: more on modeling 

# Preparing parameters
c = [5 ,6 ,8, 1, 3]
productid, profit = multidict({i: c[i] for i in range(len(c))})

# available units of material i
b = [20,40,50]
materialid, amount = multidict(zip(range(len(b)), b) )

# units of material i product j needs in order to be produced
a = np.array([[6, 7, 9, 8, 5], 
              [7, 4, 1, 9, 6], 
              [8, 3, 5, 7, 2]])
# get size of the problem
n, m = np.shape(a)
production = {}
for i in materialid:
    for j in productid:
        production[i,j] = a[i,j]

# Preparing an optimization model
mm = Model('manufacturing')

# Declaring variables
var = mm.addVars(productid, name = "product")

# Setting the objective
mm.setObjective(var.prod(profit), GRB.MAXIMIZE)

# Adding constraints
mm.addConstrs(quicksum(production[i,j]* var[j] for j in productid) <= amount[i] for i in materialid )

# Solving the optimization problem
mm.optimize()

# Printing the optimal solutions obtained
p = mm.getAttr('x', var)
print("Optimal Solutions:")
for i in productid:
    print("\nproduce product %g amount: %g" % (i, p[i]))

# Get objective value for future usage
print("\nOptimal value: \t%g" % mm.objVal)


### Production Scheduling Problem

In [7]:
# Production Scheduling Problem

# Building a model
m2 = Model('Scheduling')

# Defining decision variables
r = m2.addVars(np.arange(1,6+1), ub = 160)
v = m2.addVars(np.arange(1,6+1), ub = 50)
s = m2.addVars(np.arange(1,5+1))

# Adding constraints
m2.addConstr(r[1] + v[1] == s[1] + 105)
m2.addConstr(s[1] + r[2] + v[2] == s[2] + 170)
m2.addConstr(s[2] + r[3] + v[3] == s[3] + 230)
m2.addConstr(s[3] + r[4] + v[4] == s[4] + 180)
m2.addConstr(s[4] + r[5] + v[5] == s[5] + 150)
m2.addConstr(s[5] + r[6] + v[6] == 250)

# Setting objective
m2.setObjective(190*quicksum(r) + 260*quicksum(v) + 10*quicksum(s), GRB.MINIMIZE)

# Solving model
m2.optimize()

# Printing solutions
print("Optimal Solutions:")
for i, val in r.items(): 
    print("Number of units produced under regular hours in week %g = %g" % (i, val.getAttr("x")))
for i, val in v.items(): 
    print("Number of units produced under overtime hours in week  %g = %g" % (i, val.getAttr("x")))
for i, val in s.items(): 
    print("Number of units brought over from week %g to week %g hours s[%g] = %g" % (i, i+1, i, val.getAttr("x")))
  



Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 17 columns and 22 nonzeros
Model fingerprint: 0x3d4281f0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+02]
  Bounds range     [5e+01, 2e+02]
  RHS range        [1e+02, 2e+02]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 15 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9824180e+04   1.499973e+02   0.000000e+00      0s
       7    2.1630000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds
Optimal objective  2.163000000e+05
Optimal Solutions:
Number of units produced under regular hours in week 1 = 160
Number of units produced under regular hours in week 2 = 160
Number of units produced under regular hours in week 3 = 160
Number of units produced under regular hours in

###  Revenue Management

In [ ]:
# Revenue Management

m3 = Model('Revenue')

# Initialization
n = 5

# Expected for Q(Y) class customers from i to j
DQ = np.round(np.random.rand(n,n) * 100)
DY = np.round(np.random.rand(n,n) * 100)

# Capacity from origin i to hub
Cn0 = np.round(500*np.random.rand(n,1))

# Capacity from hub to destination j
C0n = np.round(200*np.random.rand(n,1))

# Revenue per customer in class Q(Y) from i to j
rQ = np.round(200*np.random.rand(n,n)) + 100
rY = np.round(100*np.random.rand(n,n))

# Declaring variables
Q = m3.addVars(n,n)
Y = m3.addVars(n,n)

# Setting the objective
m3.setObjective(quicksum(rQ[i,j]*Q[i,j] for i in range(n) for j in range(n)) + 
                quicksum(rY[i,j]*Y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Capacity constraints
for i in range(n):
    m3.addConstr(sum(Q[i, j] for j in range(n)) + sum(Y[i, j] for j in range(n)) <= Cn0[i])

m3.addConstrs(sum(Q[i,j] for i in range(n)) + sum(Y[i,j] for i in range(n)) <= C0n[j] for j in range(n))

# Expected demand constraints
for i in range(n):
    for j in range(n):
        m3.addConstr(Q[i,j] <= DQ[i,j])
        m3.addConstr(Y[i,j] <= DY[i,j])

# Optimize
m3.optimize()

# Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in Q.items():
    print("Number of Q class customers to accept from %g to %g: \t%g " %(i[0]+1, i[1]+1, val.getAttr("x")))
for i, val in Y.items():
    print("Number of Y class customers to accept from %g to %g: \t%g " %(i[0]+1, i[1]+1, val.getAttr("x")))
    

        
 

### Transportation Problem

In [8]:
# Transportation Problem
m4 = Model('transportation')

# Shipping costs
c = np.array([[5, 3, 2, 6],
              [7, 7, 8, 10],
              [6, 5, 3, 8]])

# Supply
s = np.array([1700, 2000, 1700])

# demand
d = np.array([1700, 1000, 1500, 1200])

# Declaring variables
var = m4.addVars(3,4)

# Setting the objective
m4.setObjective(sum(c[i,j] * var[i,j] for i in range(3) for j in range(4)), GRB.MINIMIZE)

# Demand constraints
for j in range(4):
    m4.addConstr(sum(var[i,j] for i in range(3)) == d[j])

# Supply constraints
for i in range(3):
    m4.addConstr(sum(var[i,j] for j in range(4)) <= s[i])

# Solving the optimization problem
m4.optimize()

# Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in var.items():
    print("Number of units from plant %g to outlet %g:\t %g " %(i[0]+1, i[1]+1, val.getAttr("x")))


Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x0164a657
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 2e+03]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1700000e+04   3.700000e+03   0.000000e+00      0s
       5    2.8200000e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  2.820000000e+04
Optimal Solutions:
Number of units from plant 1 to outlet 1:	 0 
Number of units from plant 1 to outlet 2:	 500 
Number of units from plant 1 to outlet 3:	 0 
Number of units from plant 1 to outlet 4:	 1200 
Number of units from plant 2 to outlet 1:	 1700 
Number of units from plant 2 t

### Scheduling Problem

In [ ]:
# Scheduling Problem
m5 = Model('scheduling')

# Demand for nurses
d = dict(zip(list(np.arange(1,8)),[80,26,33,44,55,62,71]))

# Declaring variables
x = m5.addVars(np.arange(1,8))

# Setting the objective
m5.setObjective(quicksum(x))

# Demand constraints
m5.addConstr( x[1] + x[4] + x[5] + x[6] + x[7] >= d[1])
m5.addConstr( x[1] + x[2] + x[5] + x[6] + x[7] >= d[2])
m5.addConstr( x[1] + x[2] + x[3] + x[6] + x[7] >= d[3])
m5.addConstr( x[1] + x[2] + x[3] + x[4] + x[7] >= d[4])
m5.addConstr( x[1] + x[2] + x[3] + x[4] + x[5] >= d[5])
m5.addConstr( x[2] + x[3] + x[4] + x[5] + x[6] >= d[6])
m5.addConstr( x[3] + x[4] + x[5] + x[6] + x[7] >= d[7])

# Solving the optimization problem
m5.optimize()

# Printing the optimal solutions obtained
print("Optimal Solutions:")
for i, val in x.items():
    print("Number of nurses starting their week on day %g = %g" %(i,val.getAttr("x")))

